In [1]:
text = "너 정말 병1신 같은 짓 좀 그만해. 이 ㅅ발, 왜 이렇게 일이 꼬이기만 하는 거야? 너 ㅂㅅ처럼 굴지 말고 제대로 좀 해. ㅅㅂ, 내가 왜 이딴 상황에 처하게 된 거야?"

In [2]:
slag_vocab = {
    "slangs": [
        {
            "origin": "병신",
            "meaning": "욕설"
        },
        {
            "origin": "시발",
            "meaning": "욕설"
        },        
    ]
}

slang_list = [ name['origin'] for name in slag_vocab['slangs']]
print(slang_list)

['병신', '시발']


In [4]:
import Levenshtein

# 한글 자음과 모음을 분리하는 함수
def decompose_korean(text):
    chosung = ['ㄱ', 'ㄲ', 'ㄴ', 'ㄷ', 'ㄸ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅃ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅉ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']
    jungsung = ['ㅏ', 'ㅐ', 'ㅑ', 'ㅒ', 'ㅓ', 'ㅔ', 'ㅕ', 'ㅖ', 'ㅗ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅛ', 'ㅜ', 'ㅝ', 'ㅞ', 'ㅟ', 'ㅠ', 'ㅡ', 'ㅢ', 'ㅣ']
    jongsung = ['', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

    hangul_base = 44032
    hangul_end = 55203

    decomposed_text = []
    for char in text:
        if hangul_base <= ord(char) <= hangul_end:
            code = ord(char) - hangul_base
            ch = code // 588
            j = (code - (ch * 588)) // 28
            jl = code % 28
            decomposed_text.append(chosung[ch])
            decomposed_text.append(jungsung[j])
            if jongsung[jl]:
                decomposed_text.append(jongsung[jl])
        else:
            decomposed_text.append(char)
    
    return ''.join(decomposed_text)

def levenshtein_similarity(s1, s2):
    distance = Levenshtein.distance(s1, s2)
    max_len = max(len(s1), len(s2))
    if max_len == 0:
        return 1.0  # 두 문자열이 모두 비어 있는 경우, 유사성은 1
    similarity = 1 - (distance / max_len)
    return similarity

def korean_similarity(s1, s2):
    decomposed_s1 = decompose_korean(s1)
    decomposed_s2 = decompose_korean(s2)
    
    return levenshtein_similarity(decomposed_s1, decomposed_s2)

def sliding_window(text, window_size):
    return [text[i:i+window_size] for i in range(len(text) - window_size + 1)]


# 중복 제거를 위한 함수
def remove_overlapping_results(found_words):
    # 유사성이 높은 순으로 정렬
    found_words.sort(key=lambda x: -x[3])
    idxes = []
    seen_ranges = []
    for word, window, index, similarity in found_words:
        window_range = range(index, index + len(window))
        if all(not (set(window_range) & set(existing_range)) for existing_range in seen_ranges):
            idxes.append(index)
            seen_ranges.append(window_range)
    return idxes

# 슬라이딩 윈도우 적용 및 유사성 평가
def find_similar_words(test_dialogue, slang_list, similarity_threshold=0.55):
    found_words = []
    for slang in slang_list:
        base_window_size = len(slang)
        for window_size in range(base_window_size - 2, base_window_size + 3):
            if window_size < 1:  # 윈도우 크기가 1보다 작은 경우를 방지
                continue
            windows = sliding_window(test_dialogue, window_size)
            for i, window in enumerate(windows):
                similarity = korean_similarity(slang, window)
                if similarity > similarity_threshold:
                    found_words.append((slang, window, i, similarity))
    
    # 중복 제거
    idxes = remove_overlapping_results(found_words)
    print(word, index, similarity)
    return idxes

In [6]:
find_similar_words(text, slang_list, similarity_threshold=0.5)

시발 21 0.5714285714285714


[5, 23]

'자'